In [7]:
#Global Libraries
import pandas as pd
import random
from random import seed
from random import gauss



In [8]:
def getRandomRecord(df):
    MaxRow= len(df)-1
    randValue = random.randrange(0, MaxRow, 1)
    return(df.iloc[randValue])

def usage():
    process = psutil.Process(os.getpid())
    return process.memory_info()[0] / float(2 ** 20)

def getRandomDate(startDate, endDate):
    delta = endDate - startDate
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return startDate + timedelta(seconds=random_second)


In [9]:
import gc
import os, psutil, numpy as np
import datetime
from datetime import time, timedelta


#Global variables
outputdir = 'Data\\'
sep = ';'
var_boolean = ['Y', 'N']

#Transacciones de venta - Cabecera
sah_records = 1000
sah_table = 'SALE_HEADER'
sah_fields = ['SAH_ID','CUS_ID','SAH_TIMESTAMP','STO_ID','SAH_POSNUM','SAH_TRXNUM','SAH_TOTPRODQTY','SAH_DOCTYPE'
              ,'SAH_PAYMENTTYPE','SAH_TOTPRODAMT','SAH_TOTTAX1AMT','SAH_TOTTAX2AMT']
sah_values = ['','','','','','','','','','','','']
tmp_sah_StartDate = datetime.datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
tmp_sah_endDate = datetime.datetime.strptime('2021-01-31 23:59:59', '%Y-%m-%d %H:%M:%S')
sah_trxDateRange = [tmp_sah_StartDate , tmp_sah_endDate]
tmp_sah_OpenTime = datetime.time(8, 30, 0)
tmp_sah_CloseTime = datetime.time(21, 30, 0)
sah_trxTimeRange = [tmp_sah_OpenTime, tmp_sah_CloseTime]
sah_docType = ['1', '2'] #1=BOLETA 2=FACTURA
sah_paymentType = ['1', '2', '3'] #1=Efectivo 2=Debito 3=Credito

#Transacciones de venta - Detalle
sad_ProQtyRange = [1,40]
sad_ProUnitRange = [1,10]
sad_table = 'SALE_DETAIL'
sad_fields = ['SAH_ID','SAH_TIMESTAMP','STO_ID','SAH_POSNUM','SAH_TRXNUM','PRO_ID','SAD_PRODQTY','SAD_PRODAMT'
              ,'SAD_TAX1AMT','SAD_TAX2AMT']
sad_values = ['','','','','','','','','','']


# generar archivo de Transacciones de venta
#cargar los dataframes
#previamente deben estar cargados los archivos STORE CUSTOMER y PRODUCT.
dfStore = pd.read_csv('Data\\STORE.csv', sep=";" , encoding='utf_8' )
dfProduct = pd.read_csv('Data\\PRODUCT.csv', sep=";" , encoding='utf_8')
dfCustomer = pd.read_csv('Data\\CUSTOMER.csv', sep=";" , encoding='utf_8')

#inicializa dataframes para grabar datos
df_sah = pd.DataFrame(columns = sah_fields) 
df_sad = pd.DataFrame(columns = sad_fields) 

#start timestamp
starttime = datetime.datetime.now()
print('TimeStamp: %s' % (starttime) )

tmp_listsah2 = []
tmp_listsad2 = []


#generar los registros del archivo de salida cabecera
for i in range(1,sah_records+1):
        tmp_listsah = sah_values.copy()
        df1 = getRandomRecord(dfStore)
        df2 = getRandomRecord(dfCustomer)
        tmp_listsah[0]= str(i) 
        tmp_listsah[1]= str(df2.CUS_ID)
        tmp_sahDate = getRandomDate(sah_trxDateRange[0] , sah_trxDateRange[1]).date() 
        tmp_sahStartTime= datetime.datetime.combine(tmp_sahDate, sah_trxTimeRange[0])
        tmp_sahEndTime= datetime.datetime.combine(tmp_sahDate, sah_trxTimeRange[1])
        tmp_sahTimeStamp = getRandomDate (tmp_sahStartTime,tmp_sahEndTime)
        tmp_listsah[2]= tmp_sahTimeStamp.strftime("%Y-%m-%d %H:%M:%S")
        tmp_listsah[3]= str(df1.STO_ID)
        tmp_listsah[4]= str(random.randrange(1,df1.STO_POSQTY, 1))
        tmp_listsah[5]= str(i) 
        tmp_listsah[7]= random.choice(sah_docType)
        tmp_listsah[8]= random.choice(sah_paymentType)

                      
        tmp_sah_totprodqty=random.randrange(sad_ProQtyRange[0], sad_ProQtyRange[1], 1)
        tmp_listsah[6]= str(tmp_sah_totprodqty)
        tmp_sah_totprodamt=0
        tmp_sah_tottax1amt =0
        tmp_sah_tottax2amt =0
        #generar los registros del archivo de salida detalle
        for j in range(1, tmp_sah_totprodqty+1):
            tmp_listsad = sad_values.copy()
            df3 = getRandomRecord(dfProduct)
            
            tmp_listsad[0]=str(tmp_listsah[0])
            tmp_listsad[1]=str(tmp_listsah[2])
            tmp_listsad[2]=str(tmp_listsah[3])
            tmp_listsad[3]=str(tmp_listsah[4])
            tmp_listsad[4]=str(tmp_listsah[5])
            tmp_listsad[5]=str(df3.PRO_ID)
            tmp_sadProdQty = random.randrange(sad_ProUnitRange[0], sad_ProUnitRange[1],1)
            tmp_listsad[6]=str(tmp_sadProdQty)
            tmp_sadProdAmp = tmp_sadProdQty*df3.PRO_PRICE
            tmp_listsad[7]=str(tmp_sadProdAmp)
            tmp_listsad[8]=str(int(tmp_sadProdAmp*float(df3.PRO_TAX1RATE.replace(",", "."))))
            tmp_listsad[9]=str(int(0+tmp_sadProdAmp*float(df3.PRO_TAX2RATE.replace(",", "."))))
            tmp_sah_totprodamt = tmp_sah_totprodamt + int(tmp_sadProdAmp)
            tmp_sah_tottax1amt = tmp_sah_tottax1amt + int(tmp_listsad[8])
            tmp_sah_tottax2amt = tmp_sah_tottax2amt + int(tmp_listsad[9])
            df_sad.loc[j]=tmp_listsad
                        
            
        tmp_listsah[9]= str(tmp_sah_totprodamt)
        tmp_listsah[10]= str(tmp_sah_tottax1amt)
        tmp_listsah[11]= str(tmp_sah_tottax2amt)
        df_sah.loc[i]=tmp_listsah
       
df_sad.to_csv('%s%s.csv' % (outputdir, sad_table), index=False, encoding='utf_8', sep=';', chunksize=10000)
df_sah.to_csv('%s%s.csv' % (outputdir, sah_table), index=False, encoding='utf_8', sep=';', chunksize=10000)


tmp_listsah.clear()
tmp_listsah2.clear()
tmp_listsad.clear()
tmp_listsad.clear()


#Liberar df
del [[dfStore,dfProduct,dfCustomer, df1, df2 ,df3, df_sad, df_sah]]
gc.collect()
dfStore=pd.DataFrame()
dfProduct=pd.DataFrame()
dfCustomer=pd.DataFrame()
df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_sad=pd.DataFrame()
df_sah=pd.DataFrame()
#print(usage())

#End timestamp
endtime = datetime.datetime.now()  
print('TimeStamp: %s' % (endtime) )
print(endtime - starttime)
#100 0:00:04.108813
#10000 0:00:46.668139

TimeStamp: 2022-05-11 17:33:00.827401
TimeStamp: 2022-05-11 17:33:13.824587
0:00:12.997186
